# Simple CNN notebook

## Import Libraries

In [ ]:
%matplotlib inline
import numpy as np
from utils.helper import read_data, blur, to_real, morphology
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras
FOLDER = "data/"
NUMBER_OF_RADARS = 3

## Load data

In [ ]:
data_empty = read_data(FOLDER + "empty.csv")
data_1p = read_data(FOLDER + "1p.csv")
data_2p = read_data(FOLDER + "2p.csv")
data_3p = read_data(FOLDER + "3p.csv")
print("----------------Shapes----------------")
print("empty:", data_empty.shape)
print("1p:", data_1p.shape)
print("2p:", data_2p.shape)
print("3p:", data_3p.shape)

In [ ]:
LENGTH_EMPTY = data_empty.shape[0]
LENGTH_1P = data_1p.shape[0]
LENGTH_2P = data_2p.shape[0]
LENGTH_3P = data_3p.shape[0]

In [ ]:
data_empty_summed = to_real(np.sum(data_empty, axis=1)) 
data_1p_summed = to_real(np.sum(data_1p, axis=1))
data_2p_summed = to_real(np.sum(data_2p, axis=1))
data_3p_summed = to_real(np.sum(data_3p, axis=1))

In [ ]:
data_0 = np.sum(data_empty_summed,axis = 1)
data_1 = np.sum(data_1p_summed,axis = 1)
data_2 = np.sum(data_2p_summed,axis = 1)
data_3 = np.sum(data_3p_summed,axis = 1)

In [ ]:
data_tot = np.concatenate((data_0, data_1, data_2, data_3), axis=0)
data_tot.shape

### Create Labels

In [ ]:
labels_empty = np.zeros(LENGTH_EMPTY)
labels_1p = np.ones(LENGTH_1P)
labels_2p = np.ones(LENGTH_2P) * 2
labels_3p = np.ones(LENGTH_3P) * 3
labels = np.concatenate((labels_empty, labels_1p, labels_2p, labels_3p), axis=0)
print("Labels shape:", labels.shape)

In [ ]:
train_labels_one_hot = np.eye(4)[labels.astype(int)]
print("Labels abels one hot shape:", train_labels_one_hot.shape)

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data_tot, train_labels_one_hot, test_size=0.1)
print(f"Shape train_X: {train_X.shape}")
print(f"Shape test_X: {test_X.shape}")
print(f"Shape train_y: {train_y.shape}")
print(f"Shape test_y: {test_y.shape}")

In [39]:
models = keras.models
layers = keras.layers

model = models.Sequential(
    [
        layers.Conv2D(8,
                      input_shape = (64,64,1),
                      kernel_size = 5,
                      padding = "same",
                      activation = "relu"),
        layers.MaxPooling2D(2,
                            strides = None,
                            padding = "valid"),
        layers.Conv2D(8,
                      input_shape = (32,32,1),
                      kernel_size = 3,
                      padding = "same",
                      activation = "relu"),
        layers.Flatten(),
        layers.Dense(4),
        layers.Activation("softmax")
    ]
)
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 64, 64, 8)         208       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 32, 32, 8)        0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 32, 32, 8)         584       
                                                                 
 flatten_3 (Flatten)         (None, 8192)              0         
                                                                 
 dense_3 (Dense)             (None, 4)                 32772     
                                                                 
 activation_3 (Activation)   (None, 4)                 0         
                                                      

In [40]:
model.compile(
    loss = "categorical_crossentropy",
    optimizer = keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

model.fit(
    train_X, train_y,
    epochs  = 10,
    verbose = 1,
    validation_split = 0.1,
    shuffle= True,
    callbacks=keras.callbacks.EarlyStopping(patience = 10)
)

model.save('models/CNN')

Epoch 1/10
7/7 [==============================] - 1s 43ms/step - loss: 4.8478 - accuracy: 0.2423 - val_loss: 3.5877 - val_accuracy: 0.3182
Epoch 2/10
7/7 [==============================] - 0s 24ms/step - loss: 2.2141 - accuracy: 0.5155 - val_loss: 1.0320 - val_accuracy: 0.4545
Epoch 3/10
7/7 [==============================] - 0s 26ms/step - loss: 1.5193 - accuracy: 0.4278 - val_loss: 1.0332 - val_accuracy: 0.4091
Epoch 4/10
7/7 [==============================] - 0s 34ms/step - loss: 0.9708 - accuracy: 0.6237 - val_loss: 0.7671 - val_accuracy: 0.7273
Epoch 5/10
7/7 [==============================] - 0s 25ms/step - loss: 0.8590 - accuracy: 0.6237 - val_loss: 0.7825 - val_accuracy: 0.6818
Epoch 6/10
7/7 [==============================] - 0s 24ms/step - loss: 0.6533 - accuracy: 0.7474 - val_loss: 0.6966 - val_accuracy: 0.7727
Epoch 7/10
7/7 [==============================] - 0s 26ms/step - loss: 0.8048 - accuracy: 0.6907 - val_loss: 0.5283 - val_accuracy: 0.7727
Epoch 8/10
7/7 [===========

INFO:tensorflow:Assets written to: models/CNN/assets


INFO:tensorflow:Assets written to: models/CNN/assets


In [41]:
loss, aacuracy = model.evaluate(test_X, test_y)
print(f"Loss: {loss}")
print(f"Accuracy: {aacuracy}")

1/1 [==============================] - 0s 23ms/step - loss: 0.7151 - accuracy: 0.6250
Loss: 0.7150587439537048
Accuracy: 0.625
